In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tslearn.clustering import TimeSeriesKMeans
from tslearn.preprocessing import TimeSeriesScalerMinMax
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
import joblib
import warnings
import tensorflow as tf
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
warnings.filterwarnings("ignore", category=UserWarning, module='keras')
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Reshape, Bidirectional, LSTM, Dense, Dropout, Activation
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping  # Importa EarlyStopping desde callbacks
import random


In [120]:
# Cargar y preprocesar los datos
file_path = "C:/Users/Usuario/desktop/vero2/final_dataset_completo_con_ceros.csv"
df = pd.read_csv(file_path, sep='\t')

df = df.drop('Unnamed: 0', axis=1)
df.head(12)

# filtered_df = df[df['product_id'].between(20001, 20012)]

# Opcional: Ver las primeras filas del DataFrame filtrado
# print(filtered_df.head())

# df=filtered_df

,customer_id,product_id,periodo,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,descripcion,quarter,month,close_quarter,age,mes_inicial
0,10001,20001,201812,0.0,20.0,254.62373,254.62373,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q4,12,1.0,23.0,2018-12-01
1,10001,20001,201901,0.0,53.0,393.26092,386.60688,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q1,1,0.0,24.0,2018-12-01
2,10001,20001,201902,0.0,39.0,309.90610,309.90610,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q1,2,0.0,25.0,2018-12-01
3,10001,20001,201903,0.0,23.0,142.87158,130.54927,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q1,3,1.0,26.0,2018-12-01
4,10001,20001,201904,0.0,33.0,364.37071,364.37071,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q2,4,0.0,27.0,2018-12-01
5,10001,20001,201905,0.0,31.0,439.90647,439.90647,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q2,5,0.0,28.0,2018-12-01
6,10001,20001,201906,0.0,7.0,65.92436,65.92436,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q2,6,1.0,29.0,2018-12-01
7,10001,20001,201907,0.0,14.0,144.78714,144.78714,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q3,7,0.0,30.0,2018-12-01
8,10001,20001,201908,0.0,9.0,33.63991,33.63991,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q3,8,0.0,31.0,2018-12-01
9,10001,20001,201909,0.0,18.0,111.51691,109.05244,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q3,9,1.0,32.0,2018-12-01


In [121]:
df.columns

Index(['customer_id', 'product_id', 'periodo', 'plan_precios_cuidados',
       'cust_request_qty', 'cust_request_tn', 'tn', 'cat1', 'cat2', 'cat3',
       'brand', 'sku_size', 'descripcion', 'quarter', 'month', 'close_quarter',
       'age', 'mes_inicial'],
      dtype='object')

In [ ]:
# Establecer las semillas para numpy, random y tensorflow/keras
seed_value = 42
np.random.seed(seed_value)
random.seed(seed_value)
tf.random.set_seed(seed_value)

In [122]:
# Reemplazar 082019 por promedio 07 y 09
df['periodo'] = df['periodo'].astype(str).str.strip()
df_filtered = df[df['periodo'].isin(['201907', '201908', '201909'])]
pivoted_sales = df_filtered.pivot_table(index=['product_id', 'customer_id'], columns='periodo', values='tn').reset_index()
pivoted_sales = pivoted_sales.reindex(columns=['product_id', 'customer_id', '201907', '201908', '201909'])
pivoted_sales['201908'] = pivoted_sales[['201907', '201909']].mean(axis=1)
updated_sales = pivoted_sales.melt(id_vars=['product_id', 'customer_id'], value_vars=['201907', '201908', '201909'], var_name='periodo', value_name='tn')
df.set_index(['product_id', 'customer_id', 'periodo'], inplace=True)
df.update(updated_sales.set_index(['product_id', 'customer_id', 'periodo']))
df.reset_index(inplace=True)

# Aplicar LabelEncoder a las columnas categóricas
categorical_cols = ['cat1', 'cat2', 'cat3', 'brand', 'descripcion']
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# Agrupar las ventas por periodo, cat1, cat2, cat3, brand y customer_id
df['periodo'] = pd.to_datetime(df['periodo'], format='%Y%m', errors='coerce')
grouped_df = df.groupby(['periodo', 'cat1', 'cat2', 'cat3', 'brand', 'customer_id']).agg({'tn': 'sum'}).reset_index()


# Paso 2: Calcular los ratios incluyendo customer_id
df_diciembre_2019 = df[(df['periodo'].dt.year == 2019) & (df['periodo'].dt.month == 12)]
grouped_sales_2019 = df_diciembre_2019.groupby(['cat1', 'cat2', 'cat3', 'brand', 'customer_id', 'product_id'])['tn'].sum().reset_index()
group_totals_2019 = df_diciembre_2019.groupby(['cat1', 'cat2', 'cat3', 'brand', 'customer_id'])['tn'].sum().reset_index()
ratios_2019 = pd.merge(grouped_sales_2019, group_totals_2019, on=['cat1', 'cat2', 'cat3', 'brand', 'customer_id'], suffixes=('', '_total'))
ratios_2019['ratio'] = ratios_2019['tn'] / ratios_2019['tn_total']
ratio_dict = ratios_2019.set_index(['cat1', 'cat2', 'cat3', 'brand', 'customer_id', 'product_id'])['ratio'].to_dict()

In [123]:
#pivotear el df
pivoted_df = grouped_df.pivot_table(index=['cat1', 'cat2', 'cat3', 'brand', 'customer_id'], columns='periodo', values='tn').fillna(0)

# Escalar las series temporales
scaler = TimeSeriesScalerMeanVariance(mu=0., std=1.)  # normalizar las series temporales
scaled_series = scaler.fit_transform(pivoted_df.values)
inertia = []
max_clusters = 15

# Crear un diccionario para almacenar los scalers
scalers = {}
scaled_df = grouped_df.copy()

# Aplicar StandardScaler a cada columna de interés
for col in ['tn']:
    scaler = StandardScaler()
    scaled_df[col] = scaler.fit_transform(scaled_df[[col]])
    scalers[col] = scaler

# Guardar los scalers para su uso posterior
joblib.dump(scalers, 'scalers.pkl')

['scalers.pkl']

In [125]:
# Crear el modelo TimeSeriesKMeans
n_clusters = 15
model = TimeSeriesKMeans(n_clusters=n_clusters, metric="dtw", random_state=0)

# Ajustar el modelo usando las series temporales escaladas
model.fit(scaled_series)

# Obtener los clusters asignados a cada serie temporal
clusters = model.labels_

# Añadir los clusters al DataFrame original
pivoted_df['cluster'] = clusters

# Unir el número de grupo con grouped_df
grouped_df = grouped_df.merge(pivoted_df['cluster'], left_on=['cat1', 'cat2', 'cat3', 'brand', 'customer_id'], right_index=True)


# Mostrar el DataFrame final con los números de grupo
display(grouped_df)


,periodo,cat1,cat2,cat3,brand,customer_id,tn,cluster
0,2018-12-01,0,0,4,22,10001,0.87535,0
1,2018-12-01,0,0,4,22,10002,0.27780,11
2,2018-12-01,0,0,4,22,10003,0.27256,11
3,2018-12-01,0,0,4,22,10004,0.13628,7
4,2018-12-01,0,0,4,22,10005,0.06290,12
...,...,...,...,...,...,...,...,...
434443,2019-12-01,3,13,82,32,10363,0.00000,5
434444,2019-12-01,3,13,82,32,10367,0.00000,14
434445,2019-12-01,3,13,82,32,10482,0.00000,6
434446,2019-12-01,3,13,82,32,10513,0.00000,6


In [126]:
grouped_df.to_csv("C:/Users/Usuario/desktop/vero2/grouped_df.csv", index=False)


In [127]:
#control
# Dropear la columna 'periodo'
check = grouped_df.drop(columns=['periodo'])

# Agrupar por 'cat1', 'cat2', 'cat3', 'brand', 'customer_id' y obtener el cluster asignado
result_df = check.groupby(['cat1', 'cat2', 'cat3', 'brand', 'customer_id']).agg({
    'cluster': 'first'
}).reset_index()

print(result_df)


       cat1  cat2  cat3  brand  customer_id  cluster
0         0     0     4     22        10001        0
1         0     0     4     22        10002       11
2         0     0     4     22        10003       11
3         0     0     4     22        10004        7
4         0     0     4     22        10005       12
...     ...   ...   ...    ...          ...      ...
40378     3    13    82     32        10363        5
40379     3    13    82     32        10367       14
40380     3    13    82     32        10482        6
40381     3    13    82     32        10513        6
40382     3    13    82     32        10552       12

[40383 rows x 6 columns]


In [128]:
# Agrupar por 'cat1', 'cat2', 'cat3', 'brand', 'customer_id' y contar clusters únicos
cluster_counts = check.groupby(['cat1', 'cat2', 'cat3', 'brand', 'customer_id']).agg({
    'cluster': 'nunique'
}).reset_index()

# Filtrar combinaciones con más de un cluster
multiple_clusters = cluster_counts[cluster_counts['cluster'] > 1]

# Mostrar combinaciones con más de un cluster
print(multiple_clusters)


Empty DataFrame
Columns: [cat1, cat2, cat3, brand, customer_id, cluster]
Index: []


hasta aca ok con grouped_df guardado en vero2

In [129]:
# Lista para acumular las filas de resultados
resultados_por_producto = []

# Elegir un número de pasos de tiempo
n_steps = 13  # Ventana de tiempo de 18 meses
n_features = 6 # Cambia esto si tienes más características
step_ahead= 2

def crear_secuencias(datos, n_steps, step_ahead=1):
    X, y = [], []
    for i in range(len(datos) - n_steps - step_ahead + 1):
        end_ix = i + n_steps
        out_end_ix = end_ix + step_ahead - 1
        seq_x, seq_y = datos[i:end_ix], datos[out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)



# Definir la función para construir el modelo
def build_model(input_shape):
    model = Sequential()
    model.add(Bidirectional(LSTM(units=256, return_sequences=True), input_shape=(n_steps, n_features)))
    model.add(Dropout(0.3))
    model.add(Bidirectional(LSTM(units=128)))
    model.add(Dropout(0.3))
    model.add(Dense(64))
    model.add(Activation('tanh'))
    model.add(Dense(1))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss='mse')
    return model

# Inicializar EarlyStopping
early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

In [130]:
# Reescalar 'tn' en grouped_df
scaler = StandardScaler()
grouped_df['tn_scaled'] = scaler.fit_transform(grouped_df[['tn']])


ahora tengo grouped_df con escalado en tn: se agrega columna tn_scaled

In [131]:
grouped_df.head()

,periodo,cat1,cat2,cat3,brand,customer_id,tn,cluster,tn_scaled
0,2018-12-01,0,0,4,22,10001,0.87535,0,-0.003176
1,2018-12-01,0,0,4,22,10002,0.27780,11,-0.082939
2,2018-12-01,0,0,4,22,10003,0.27256,11,-0.083639
3,2018-12-01,0,0,4,22,10004,0.13628,7,-0.101830
4,2018-12-01,0,0,4,22,10005,0.06290,12,-0.111625


In [ ]:
# ver de dejar solo la fecha y tn y cluster

In [132]:
# Seleccionar solo las columnas 'periodo', 'tn' y 'cluster'
time_series_df = grouped_df[['periodo', 'tn_scaled', 'cluster']]

# Mostrar el resultado
print(time_series_df)


          periodo  tn_scaled  cluster
0      2018-12-01  -0.003176        0
1      2018-12-01  -0.082939       11
2      2018-12-01  -0.083639       11
3      2018-12-01  -0.101830        7
4      2018-12-01  -0.111625       12
...           ...        ...      ...
434443 2019-12-01  -0.120021        5
434444 2019-12-01  -0.120021       14
434445 2019-12-01  -0.120021        6
434446 2019-12-01  -0.120021        6
434447 2019-12-01  -0.120021       12

[434448 rows x 3 columns]


OJO DEJE EPOCHS EN 100!!!!!!!!!!!!!!!!!!!!!!

In [133]:
from keras.callbacks import ReduceLROnPlateau
import numpy as np
import pandas as pd

models = {}
predictions = []

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=0.00001, verbose=1)

# Preparar los datos por cluster
for cluster in range(n_clusters):
    print(f'Entrenando cluster numero: {cluster}')
    cluster_data = grouped_df[grouped_df['cluster'] == cluster].copy()
    cluster_data.sort_values(by='periodo', inplace=True)
    
    # Preparar los datos para LSTM
    X, y = [], []
    for key, data in cluster_data.groupby(['cat1', 'cat2', 'cat3', 'brand', 'customer_id']):
        series = data[['cat1', 'cat2', 'cat3', 'brand', 'customer_id', 'tn_scaled']].values
        if len(series) > 2:  # Asegurarse de que haya suficientes datos
            X.append(series[:-2])  # Todos los datos excepto los últimos 2
            y.append(series[-1, 0])  # Selecciona solo tn_scaled como objetivo

    # Pad y reshape de X
    max_len = max(len(seq) for seq in X)
    X_padded = np.array([np.pad(seq, ((max_len - len(seq), 0), (0, 0)), mode='constant') for seq in X]).astype(np.float32)
    y = np.array(y).astype(np.float32)
    
    if len(X_padded) == 0 or len(y) == 0:
        continue

    # Construir y entrenar el modelo
    model = build_model((X_padded.shape[1], X_padded.shape[2]))
    model.fit(X_padded, y, epochs=100, verbose=2, batch_size=128, validation_split=0.2, callbacks=[reduce_lr])
    models[cluster] = model

    # Hacer predicciones
    for key, data in cluster_data.groupby(['cat1', 'cat2', 'cat3', 'brand', 'customer_id']):
        series = data[['cat1', 'cat2', 'cat3', 'brand', 'customer_id', 'tn_scaled']].values
        
        if len(series) > 2:
            max_len = len(series) - 1
            X_pred = np.pad(series[1:], ((max_len - len(series[1:]), 0), (0, 0)), mode='constant').astype(np.float32)
            X_pred = np.reshape(X_pred, (1, X_pred.shape[0], X_pred.shape[1]))
            pred = model.predict(X_pred, verbose=0)
            predictions.append([key[0], key[1], key[2], key[3], pred[0][0], key[4]])

    # Guardar las predicciones en un DataFrame y exportar a CSV
    pred_df_temp = pd.DataFrame(predictions, columns=['cat1', 'cat2', 'cat3', 'brand', 'prediccion', 'customer_id'])
    pred_df_temp.to_csv(f"predicciones_temporales_cluster_{cluster}.csv", index=False)


Entrenando cluster numero: 0
Epoch 1/100
10/10 - 9s - 948ms/step - loss: 1.4013 - val_loss: 0.5062 - learning_rate: 0.0010
Epoch 2/100
10/10 - 1s - 133ms/step - loss: 0.8274 - val_loss: 1.6473 - learning_rate: 0.0010
Epoch 3/100
10/10 - 1s - 103ms/step - loss: 0.7116 - val_loss: 0.9295 - learning_rate: 0.0010
Epoch 4/100
10/10 - 1s - 91ms/step - loss: 0.7202 - val_loss: 0.9725 - learning_rate: 0.0010
Epoch 5/100
10/10 - 1s - 87ms/step - loss: 0.7323 - val_loss: 1.4526 - learning_rate: 0.0010
Epoch 6/100
10/10 - 1s - 101ms/step - loss: 0.7348 - val_loss: 1.0228 - learning_rate: 0.0010
Epoch 7/100
10/10 - 1s - 87ms/step - loss: 0.6965 - val_loss: 1.3115 - learning_rate: 0.0010
Epoch 8/100
10/10 - 1s - 89ms/step - loss: 0.7019 - val_loss: 1.0155 - learning_rate: 0.0010
Epoch 9/100
10/10 - 1s - 86ms/step - loss: 0.6939 - val_loss: 1.5243 - learning_rate: 0.0010
Epoch 10/100
10/10 - 1s - 79ms/step - loss: 0.7516 - val_loss: 0.8482 - learning_rate: 0.0010
Epoch 11/100

Epoch 11: ReduceLROnPl

In [134]:
pred_df_temp.head()

,cat1,cat2,cat3,brand,prediccion,customer_id
0,0,0,4,22,0.97414,10001
1,0,0,4,22,0.97414,10032
2,0,0,4,22,0.97414,10106
3,0,0,4,22,0.97414,10144
4,0,0,4,22,0.97414,10225


In [135]:
pred_df_temp.to_csv("C:/Users/Usuario/desktop/vero2/pred_df_temp.csv", index=False)


In [136]:
pred_df = pd.DataFrame(predictions, columns=['cat1', 'cat2', 'cat3', 'brand', 'prediccion', 'customer_id'])
# Suponiendo que `scaler` es el objeto StandardScaler que usaste para escalar `tn` originalmente

# Desescalar las predicciones
predicciones_desescaladas = scaler.inverse_transform(pred_df['prediccion'].values.reshape(-1, 1))

# Reemplazar las predicciones escaladas con las desescaladas en el DataFrame
pred_df['prediccion'] = predicciones_desescaladas.flatten()

# Ahora pred_df tiene las predicciones desescaladas en la columna 'prediccion'
pred_df.head()


,cat1,cat2,cat3,brand,prediccion,customer_id
0,0,0,4,22,8.196926,10001
1,0,0,4,22,8.196926,10032
2,0,0,4,22,8.196926,10106
3,0,0,4,22,8.196926,10144
4,0,0,4,22,8.196926,10225


In [137]:
# Obtener combinaciones únicas de 'product_id', 'cat1', 'cat2', 'cat3', 'brand' del DataFrame original
distinct_combinations = df[['product_id', 'cat1', 'cat2', 'cat3', 'brand']].drop_duplicates()

# Realizar un join con pred_df en las columnas correspondientes
pred_df = pred_df.merge(distinct_combinations, on=['cat1', 'cat2', 'cat3', 'brand'], how='left')

# Mostrar el DataFrame con las predicciones y las nuevas columnas agregadas
print(pred_df.head())


   cat1  cat2  cat3  brand  prediccion  customer_id  product_id
0     0     0     4     22    8.196926        10001       20609
1     0     0     4     22    8.196926        10032       20609
2     0     0     4     22    8.196926        10106       20609
3     0     0     4     22    8.196926        10144       20609
4     0     0     4     22    8.196926        10225       20609


In [138]:
# Inicializar una lista para almacenar las predicciones ajustadas
predictions_adjusted = []

# Iterar sobre las filas de pred_df
for index, row in pred_df.iterrows():
    key = (row['cat1'], row['cat2'], row['cat3'], row['brand'], row['product_id'])
    
    # Buscar el ratio correspondiente en ratio_dict utilizando la clave correcta
    if key in ratio_dict:
        ratio = ratio_dict[key]
        # Calcular la predicción ajustada
        prediccion_ajustada = row['prediccion'] * ratio
        predictions_adjusted.append(prediccion_ajustada)
    else:
        predictions_adjusted.append(row['prediccion'])  # Mantener la predicción original si no hay ratio definido

# Agregar las predicciones ajustadas al DataFrame pred_df
pred_df['prediccion_ajustada'] = predictions_adjusted

# Mostrar el DataFrame con las predicciones ajustadas
print(pred_df.head())


   cat1  cat2  cat3  brand  prediccion  customer_id  product_id  \
0     0     0     4     22    8.196926        10001       20609   
1     0     0     4     22    8.196926        10032       20609   
2     0     0     4     22    8.196926        10106       20609   
3     0     0     4     22    8.196926        10144       20609   
4     0     0     4     22    8.196926        10225       20609   

   prediccion_ajustada  
0             8.196926  
1             8.196926  
2             8.196926  
3             8.196926  
4             8.196926  


In [139]:
# Sumarizar las predicciones ajustadas por product_id
summarized_predictions = pred_df.groupby('product_id')['prediccion_ajustada'].sum().reset_index()

# Mostrar el DataFrame con las predicciones sumarizadas
print(summarized_predictions)


     product_id  prediccion_ajustada
0         20001          4254.694915
1         20002          4301.131234
2         20003          3624.426756
3         20004          3624.426756
4         20005          3624.426756
..          ...                  ...
775       21263           383.534721
776       21265          1085.767240
777       21266          1085.767240
778       21267           228.564031
779       21276           228.564031

[780 rows x 2 columns]


In [140]:
# Guardar las predicciones finales en un archivo CSV
summarized_predictions.to_csv("C:/Users/Usuario/desktop/vero2/modelodtw2000.csv", index=False)